### Image Classification Model (.h5) Generation for Go or No Go (Sample Example)
## Uses images from **IBM COS (Bucket)** labeled using **IBM Cloud Annotations**


# Setup

In [ ]:
credentials = {
  "bucket": "<bucket_name>",
  "access_key_id": "<access_key_id>",
  "secret_access_key": "<secret_access_key>",
  "endpoint_url": "<endpoint_url>"
}

## Install Prerequisites

We use Keras/Tensorflow to build the classification model, and visualize the process with matplotlib.

In [ ]:
!pip install tensorflow==2.5.0 ibm-cos-sdk==2.6 h5py==2.10.0

In [ ]:
# Import required libraries
import os
import uuid
import shutil
import json

import ibm_boto3
import tensorflow as tf
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np

## Read The Data

Here we build simple wrapper functions to read in the data from our cloud object storage buckets and extract it.

A download function from IBM Cloud

In [ ]:
def download_file_cos(credentials, local_file_name, key): 
    '''
    Wrapper function to download a file from cloud object storage using the
    credential dict provided and loading it into memory
    '''
    cos = ibm_boto3.client(
        service_name='s3',
        aws_access_key_id=credentials['access_key_id'],
        aws_secret_access_key=credentials['secret_access_key'],
        endpoint_url=credentials['endpoint_url'])
    try:
        res=cos.download_file(Bucket=credentials['bucket'], Key=key, Filename=local_file_name)
    except Exception as e:
        print(Exception, e)
    else:
        print('File Downloaded')

def get_annotations(credentials): 
    cos = ibm_boto3.client(
        service_name='s3',
        aws_access_key_id=credentials['access_key_id'],
        aws_secret_access_key=credentials['secret_access_key'],
        endpoint_url=credentials['endpoint_url'])
    try:
        return json.loads(cos.get_object(Bucket=credentials['bucket'], Key='_annotations.json')['Body'].read())
    except Exception as e:
        print(Exception, e)

In [ ]:
base_path = 'data'
if os.path.exists(base_path) and os.path.isdir(base_path):
    shutil.rmtree(base_path)
os.makedirs(base_path, exist_ok=True)

annotations = get_annotations(credentials)

for i, image in enumerate(annotations['annotations'].keys()):
    label = annotations['annotations'][image][0]['label']
    os.makedirs(os.path.join(base_path, label), exist_ok=True)
    _, extension = os.path.splitext(image)
    local_path = os.path.join(base_path, label, str(uuid.uuid4()) + extension)
    download_file_cos(credentials, local_path, image)

In [ ]:
print("NO GO:")
!ls data/nogo
print("GO:")
!ls data/go

## Build the Model

We start with a [MobileNetV2](https://arxiv.org/abs/1801.04381) architecture as the backbone [pretrained feature extractor](https://github.com/tensorflow/models/tree/master/research/slim/nets/mobilenet). We then add a couple of dense layers and a softmax layer to perfom the classification. We freeze the MobileNetV2 backbone with weights trained on ImageNet dataset and only train the dense layers and softmax layer that we have added.

Configuration needs to change depending on the image size and aspect ratio. Or you might use other backbone or libraries for training and exporting model.

In [ ]:
base_model=tf.keras.applications.MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3), alpha=1.0) #imports the mobilenet model and discards the last 1000 neuron layer.
x=base_model.output
x=tf.keras.layers.GlobalAveragePooling2D()(x)
x=tf.keras.layers.Dense(512,activation='relu')(x) #dense layer 1
x=tf.keras.layers.Dense(256,activation='relu')(x) #dense layer 2
preds=tf.keras.layers.Dense(3,activation='softmax')(x) #final layer with softmax activation

model=tf.keras.Model(inputs=base_model.input,outputs=preds)


In [ ]:
#Freeze layers from MobileNetV2 backbone (not to be trained)
for layer in base_model.layers:
    layer.trainable=False

In [ ]:
#Prepare the training dataset as a data generator object
train_datagen=tf.keras.preprocessing.image.ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input) #included in our dependencies

train_generator=train_datagen.flow_from_directory('data',
                                                 target_size=(224,224),
                                                 color_mode='rgb',
                                                 batch_size=10,
                                                 class_mode='categorical',
                                                 shuffle=True)

#### Using Adam, categorical_crossentropy and accuracy as optimization method, loss function and metrics, respectively

In [ ]:
# Build the model
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()

## Train the model

In [ ]:
from tensorflow.random import set_seed
set_seed(3)
step_size_train=1
epochs=1
log_file = model.fit(train_generator,
                   steps_per_epoch=step_size_train,
                   epochs=epochs)

## Figure of Training Loss and Accuracy
Plotting the training result as below (if required)

In [ ]:
# # Model accuracy and loss vs epoch
# plt.plot(log_file.history['acc'], '-bo', label="train_accuracy")
# plt.plot(log_file.history['loss'], '-r*', label="train_loss")
# plt.title('Training Loss and Accuracy')
# plt.ylabel('Loss/Accuracy')
# plt.xlabel('Epoch #')
# plt.legend(loc='center right')
# plt.show()

## Model Performance

Here we perform inference on some sample data points to determine the performance of the model

In [ ]:
# Mapping labels 
label_map = (train_generator.class_indices)

In [ ]:
label_map
# Returns Example: {'go': 0, 'nogo': 1}

In [ ]:
# Creating a sample inference function
def prediction(image_path, model):
    img = tf.keras.preprocessing.image.load_img(image_path, target_size=(224, 224))
    x = tf.keras.preprocessing.image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = tf.keras.applications.mobilenet_v2.preprocess_input(x)
    preds = model.predict(x)
    # print('Predictions', preds)
    
    #  Printing the prediction score and class
    # for pred, value in label_map.items():    
    #     if value == np.argmax(preds):
    #         print('Predicted class is:', pred)
    #         print('With a confidence score of: ', np.max(preds))

    # Format the Output as required by ISAC-SIMO Offline Model (Classifier)
    # https://www.isac-simo.net/app/offline_model/readme.md#classifier-without-pre-post-process
    if np.argmax(preds) == 0:
      return [[np.max(preds), 0]]
    elif np.argmax(preds) == 1:
      return [[0, np.max(preds)]]
    else:
      return [[0, 0]]

In [ ]:
# Download some Sample Images (Example: Stirrup)
go_url = 'https://upload.wikimedia.org/wikipedia/commons/thumb/c/cf/Stirrup_sample.png/220px-Stirrup_sample.png'
nogo_url = 'https://www.zbms.co.zw/wp-content/uploads/2018/06/230-by-230-rebar-stirrups-links-for-sale-in-Harare-Zimbabwe-1.jpg'
!wget {go_url} -O go.jpg 
!wget {nogo_url} -O nogo.jpg

In [ ]:
# Opening test image
image = Image.open("go.jpg")
image

In [ ]:
prediction('go.jpg', model)
# Returns Example: [[0.69597286, 0]]

In [ ]:
# Generate and Download the .h5 Model
from google.colab import files
!mkdir -p saved_model
model.save('saved_model/new-ibm-test.h5')
files.download('saved_model/new-ibm-test.h5')